In [ ]:
# import datasets
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2023-04-07 04:35:44--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 404 The specified resource does not exist.
2023-04-07 04:35:45 ERROR 404: The specified resource does not exist..



In [ ]:
!ls

drive  README_rpc-cv-images.txt  rockpaperscissors.zip	runs	     scissors
paper  rock			 rps-cv-images		sample_data


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [3]:
# !kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset
!kaggle datasets download -d drgfreeman/rockpaperscissors


 99% 304M/306M [00:10<00:00, 30.6MB/s]
100% 306M/306M [00:10<00:00, 31.9MB/s]


In [4]:
from zipfile import ZipFile
file_name = "/content/rockpaperscissors.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

  

Done


In [5]:
import os

file_path = 'rps-cv-images/paper/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'rps-cv-images/rock/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'rps-cv-images/scissors/Thumbs.db'
if os.path.exists(file_path):
    os.remove(file_path)

file_path = 'content/rps-cv-images/README_rpc-cv-images.txt'
if os.path.exists(file_path):
    os.remove(file_path)
    

In [ ]:
os.path.exists('content/rps-cv-images/README_rpc-cv-images.txt')

In [ ]:
!ls

In [70]:
import torch
import numpy as np
import torch.nn
from PIL import Image
from torchvision import transforms
import os
from sklearn.preprocessing import LabelEncoder
import random
from torch.utils.data import Dataset
from zipfile import ZipFile
class custom_dataset(Dataset):

    # initialize your dataset class
    def __init__(self, mode="train", tr=None, image_path="rps-cv-images", label_path="rps-cv-images"):
        self.mode = mode
        self.image_path = image_path
        self.label_path = label_path
        self.total_images = []
        self.labels = []
        self.papers = []
        self.rocks = []
        self.scissorss = []
        self.tr = tr
        self.unique_labels = []
        # self.train_list = []
        # self.train_labels = []

        # self.val_list = []
        # self.val_labels = []

        # self.test_list = []
        # self.test_labels = []

        # self.unique_labels = []

        # for i in os.listdir(image_path + "train/"):
        #     self.unique_labels.append(i)
        #     for j in os.listdir(image_path+"train/" + i):
        #         self.train_list.append(i+'/'+j)
        #         self.train_labels.append(i)

        # for i in os.listdir(image_path + "test/"):

        #     for j in os.listdir(image_path+"test/" + i):
        #         self.test_list.append(i+'/'+j)
        #         self.test_labels.append(i)

        # for i in os.listdir(image_path + "val/"):

        #     for j in os.listdir(image_path+"val/" + i):
        #         self.val_list.append(i+'/'+j)
        #         self.val_labels.append(i)

        for i in os.listdir(image_path):
            class_balance = []
            self.unique_labels.append(i)
            for j in os.listdir(image_path+"/" + i):
                class_balance.append(image_path+"/"+i+"/"+j)

                self.labels.append(i)
                self.total_images.append(i+'/'+j)
            if str.lower(i) == 'paper':
                self.papers = class_balance
            elif str.lower(i) == 'rock':
                self.rocks = class_balance
            elif str.lower(i) == 'scissors':
                self.scissorss = class_balance
        print("check if dataset is balanced: ")
        print(len(self.papers), len(self.rocks),len(self.scissorss))
        print(len(self.total_images))
        print(len(self.labels))
        self.unique_labels.remove('.ipynb_checkpoints')
        print(self.unique_labels)
        
        
        
        
        # # encode the labels as integers
        # self.label_encoder = LabelEncoder()
        # self.labels = self.label_encoder.fit_transform(self.labels)
        
        np.random.seed(42)

        # Combine the image paths and labels into arrays
        total_images = np.array(self.total_images)
        labels = np.array(self.labels)
        # print(type(total_images))
        # print(type(labels))
        # Generate a random permutation of the indices
        
        indices = np.random.permutation(len(total_images))

        # Shuffle the images and labels arrays using the permuted indices
        shuffled_images = total_images[indices]
        shuffled_labels = labels[indices]
        # Shuffle the images and labels using a random state
        #distribute to val train and test





        train_size = 1908
        val_size = 88
        test_size = 192


        # Split the dataset into training (22,000), testing (2,000), and validation (1,000) sets
        train_images = shuffled_images[:train_size]
        train_labels=shuffled_labels[:train_size]

        test_images = shuffled_images[train_size:train_size+test_size]
        test_labels = shuffled_labels[train_size:train_size+test_size]

        val_images = shuffled_images[train_size+test_size:]
        val_labels = shuffled_labels[train_size+test_size:]

        


    
        # # Combine the image paths and labels into tuples
        # data = list(zip(self.total_images, self.labels))

        # # Shuffle the data randomly
        # random.shuffle(data)

        # # Split the dataset into training (22,000), testing (2,000), and validation (1,000) sets
        # train_data = data[:train_size]
        # test_data = data[train_size:train_size+test_size]
        # val_data = data[train_size+test_size:]

        # # Extract the image paths and labels from the resulting sets
        # train_images, train_labels = zip(*train_data)
        # test_images, test_labels = zip(*test_data)
        # val_images, val_labels = zip(*val_data)




        if(self.mode == "train"):
            self.image_list = train_images
            self.labels = train_labels
        elif(self.mode == "val"):
            self.image_list = val_images
            self.labels = val_labels
        elif(self.mode == "test"):
            self.image_list = test_images
            self.labels = test_labels
        # print(type(self.image_list))
        # print(type(self.labels))

        
        # labeller ve sekillerden 2000 test 1000 val 22000 train
        # hem labeller hem sekilleri

        

    def __getitem__(self, index):
        

        image = Image.open(self.image_path+'/'+self.image_list[index])
        label = self.labels[index]
        # print(label)
        # print(self.unique_labels)
        label = self.parse_labels(label)
        # print(label)
        # Convert the image to RGB scale
        image = image.convert('RGB')
        
        # image.show()
        
        # label = self.parse_labels(label)
        # label = [0, 1]
        if(self.tr):
            image = self.tr(image)

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        # Create a transformation that normalizes the image
        transform = transforms.Compose([
            transforms.Resize(
            (224, 224), interpolation=transforms.InterpolationMode.NEAREST),# Resize the image to 224x224
            transforms.CenterCrop(size=(224, 224)),   
            transforms.ToTensor(),           # Convert the image to a tensor
            transforms.Normalize(mean, std)  # Normalize the image
        ])

        
        
        image = transform(image)
        

        return image, label
    
    
    def parse_labels(self, label):
        arr = np.zeros((len(self.unique_labels),), dtype= float)
        for i in range(len(self.unique_labels)):
            if label == self.unique_labels[i]:
                arr[i] = 1.0
        return arr
    


    def __len__(self):
        return len(self.image_list)
        # return None


custom_dataset()[10]
# print(self.label)
# cat is 0 and dog is 1


check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']


(tensor([[[-1.4843, -1.5014, -1.5014,  ..., -2.0323, -1.9638, -1.8439],
          [-1.5357, -1.5699, -1.5870,  ..., -2.0152, -1.9980, -1.8953],
          [-1.5357, -1.5699, -1.6042,  ..., -2.0152, -2.0323, -1.9295],
          ...,
          [-1.5699, -1.6042, -1.5699,  ..., -1.6042, -1.6042, -1.5357],
          [-1.5870, -1.6042, -1.5699,  ..., -1.6042, -1.6042, -1.5357],
          [-1.5870, -1.6042, -1.6042,  ..., -1.6042, -1.5699, -1.5357]],
 
         [[ 0.1001,  0.0826,  0.0826,  ..., -1.8431, -1.7731, -1.6506],
          [ 0.1352,  0.1176,  0.1176,  ..., -1.7731, -1.7556, -1.6506],
          [ 0.1352,  0.1176,  0.1001,  ..., -1.7731, -1.8081, -1.6856],
          ...,
          [ 0.1001,  0.1001,  0.1001,  ...,  0.1527,  0.1527,  0.1176],
          [ 0.0826,  0.1001,  0.1001,  ...,  0.1527,  0.1527,  0.1176],
          [ 0.1176,  0.1001,  0.1176,  ...,  0.1527,  0.1352,  0.1176]],
 
         [[-1.0550, -1.0724, -1.0724,  ..., -1.5953, -1.5430, -1.4210],
          [-1.0550, -1.1073,

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# images = images.to(device)
print(device)

cuda


# Model

In [71]:

# import torch
# import torch.nn as nn


# class ExModel(nn.Module):

#     def __init__(self, n_classes):
#         super().__init__()

#         # as images are in RGB, they have originally 3 channels,
#         # all channels should be speicified beforehand
#         self.conv1 = nn.Conv2d(in_channels=3, kernel_size=(
#             3, 3), out_channels=128)
#         self.relu = nn.ReLU()
#         self.max_pooling = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
#         # 2ci conc=volutional layer yaratmaq
#         self.conv2 = nn.Conv2d(in_channels=128, kernel_size=(
#             3, 3), out_channels=256)  # kernel size filetr size
#         # Max poolingle biz sizeni azaldiriq

#         self.flatten = nn.Flatten()  # hamsini bir arraye saliriq
#         # 256 is out channel errorda ozu verecek size, matrixin dogru hesablanmis halini verecek
#         self.classifier_layer1 = nn.Linear(
#             in_features=746496, out_features=128)
#         self.classifier_layer2 = nn.Linear(
#             in_features=128, out_features=n_classes)
        
#         # out =128 , novbeti input 128dir out=n_classes

#     def forward(self, image):
#         x = self.conv1(image)
#         x = self.relu(x)
#         x = self.max_pooling(x)
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.max_pooling(x)
#         x = self.flatten(x)
#         pred = self.classifier_layer1(x)  # pred
#         pred = self.relu(pred)
#         # pred = self.flatten(pred)
#         pred = self.classifier_layer2(pred)
#         # Apply softmax function to convert output to probability distribution
#         # pred = nn.functional.softmax(pred, dim=1)
#         # pred=nn.functional.relu(pred)
#         # pred = self.classifier_layer(pred)
#         # pred = nn.functional.softmax(pred,dim=1)

#         return pred

# model = ExModel(2).to(device)
# print(model)

# VGG16 model
import torch
import torch.nn as nn


class ExModel(nn.Module):

    def __init__(self, n_classes):
        super().__init__()

        # as images are in RGB, they have originally 3 channels,
        # all channels should be specified beforehand
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(in_features=512 * 7 * 7, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=n_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


# model = ExModel(3).to(device)
# print(model)


In [9]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 23.5 MB/s eta 0:00:00


In [72]:
# #utils file
# from torchmetrics.classification import MulticlassF1Score
# import torch


# def metrics(preds, target,device):
#     metr = MulticlassF1Score(num_classes=2).to(device)
#     preds=preds.to(device)
#     target = target.to(device)
#     return metr(preds, target)


from torchmetrics.classification import MulticlassF1Score
import torch


def metrics(preds, target, device):
    metr = MulticlassF1Score(num_classes=3).to(device)
    preds = preds.to(device)
    target = target.to(device)
    return metr(preds, target)


In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# %env TORCH_USE_CUDA_DSA=1


In [73]:
# train file
import torch
import torch.nn as nn
from torch.nn.functional import softmax

from torch.utils.data import DataLoader, Dataset
# from models.example_model import ExModel
# from example_model import ExModel
# from datasets.dataset_retrieval import custom_dataset
from torch.optim import SGD

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm as progress_bar
# from utils import metrics

import os

save_model_path = "checkpoints/"
pth_name = "saved_model.pth"


def val(model, data_val, loss_function, writer, epoch):

    f1_score = 0
    data_iterator = enumerate(data_val)  # take batches
    with torch.no_grad():

        model.eval()  # switch model to evaluation mode
        tq =progress_bar(total=len(data_val))
        tq.set_description('Validation:')

        total_loss = 0

        for _, batch in data_iterator:
            # forward propagation
            images, labels = batch
            # images,labels=images.to(device),labels.to(device)
            

            # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            # model.to(device)
            # images = images.to(device)

            pred = model(images.to(device))
            # pred = model(image.cuda())
            pred = softmax(pred, dim=1)
            # loss = loss_function(pred.cuda(), label.cuda())
            loss = loss_function(pred, labels.to(device))

            # softmax mes 53 netiemiz var onlari ele formta salir ki her biri mehz 1-in nece faizini teskil edir.hamsini ele formta salir ki umumi cemleri 1 olur
            # pred = pred.softmax(dim=1)
            f1_score += metrics(pred, labels.to(device),device)
            # f1_score += metrics(pred, label.cuda())
            total_loss += loss.item()
            tq.update(1)

    writer.add_scalar("Validation mIoU", f1_score/len(data_val), epoch)
    writer.add_scalar("Validation Loss", total_loss/len(data_val), epoch)

    
    tq.close()
    print("F1 score: ", f1_score/len(data_val))
    print("Total Loss: ", total_loss/len(data_val))

    return None


def train(model, train_data, val_data,  optimizer, loss, max_epoch):

    writer = SummaryWriter()
    for epoch in range(max_epoch):

        # if you are going to update your model, put it in train mode.
        model.train()

        f1_score = 0  # to find total performance per epoch
        loss_total = 0

        data_iterator = enumerate(train_data)

        # tqdm is library to see he progressbar
        tq = progress_bar(total=len(train_data))
        tq.set_description('epoch %d' % (epoch))

        for it, batch in data_iterator:
            optimizer.zero_grad()
            images, labels = batch
            # labels=labels.to(device)

            

            # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            # model.to(device)
            # images = images.to(device)

            pred = model(images.to(device))
            # pred = model(images.cuda())

           
            pred = softmax(pred, dim=1)
            loss_value = loss(pred, labels.to(device))
            f1_score += metrics(pred, labels.to(device),device)
            # loss_value = loss(pred, labels.cuda())
            # f1_score += metrics(pred, labels.cuda())
            loss_value.backward()
            optimizer.step()

            # pay attention!! if you dont write .item() you will overload gpu
            loss_total += loss_value.item()

            tq.set_postfix(loss_st='%.6f' % loss_value)
            tq.update(1)

        writer.add_scalar("Training F1", f1_score/len(train_data), epoch)
        writer.add_scalar("Training Loss", loss_total/len(val_data), epoch)

        tq.close()

        val(model, val_data, loss, writer, epoch)

        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(checkpoint, os.path.join(save_model_path, pth_name))
        print("saved the model " + save_model_path)


train_data = custom_dataset("train")
val_data = custom_dataset("val")



train_loader = DataLoader(
    train_data,
    batch_size=32,  # 1 sekil verecekdir
    shuffle=True
)

val_loader = DataLoader(
    val_data,
    batch_size=32
)


model = ExModel(3).to(device)
# print(model(train_data[0][0]).unsqueeze(dim=1))
optimizer = SGD(model.parameters(),  lr=0.001)

loss = nn.CrossEntropyLoss()


# if you want to load your pretrained model or
# you want to resume stopped training
# use torch.load_state_dict by checking the library!


train(model, train_loader, val_loader, optimizer,loss, 5)  # max number of epochs, full scan of dataset
# batch normalization


check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']
check if dataset is balanced: 
712 726 750
2188
2188
['rock', 'paper', 'scissors']



epoch 0: 100%|██████████| 60/60 [00:34<00:00,  1.73it/s, loss_st=1.098636]

Validation:: 100%|██████████| 3/3 [00:00<00:00,  3.71it/s]


F1 score:  tensor(0.2667, device='cuda:0')
Total Loss:  1.0983396723038619
saved the model checkpoints/



epoch 1: 100%|██████████| 60/60 [00:33<00:00,  1.79it/s, loss_st=1.098617]

Validation:: 100%|██████████| 3/3 [00:00<00:00,  3.93it/s]


F1 score:  tensor(0.2667, device='cuda:0')
Total Loss:  1.098347082734108
saved the model checkpoints/


...

F1 score:  tensor(0.2667, device='cuda:0')
Total Loss:  1.0983549112247095
saved the model checkpoints/



epoch 3: 100%|██████████| 60/60 [00:33<00:00,  1.78it/s, loss_st=1.098774]

Validation:: 100%|██████████| 3/3 [00:00<00:00,  3.52it/s]


F1 score:  tensor(0.2667, device='cuda:0')
Total Loss:  1.0983625977403586
saved the model checkpoints/



epoch 4: 100%|██████████| 60/60 [00:33<00:00,  1.77it/s, loss_st=1.098972]

Validation:: 100%|██████████| 3/3 [00:00<00:00,  3.95it/s]


F1 score:  tensor(0.2667, device='cuda:0')
Total Loss:  1.0983705098430316
saved the model checkpoints/


In [13]:

###################
# Load saved model
saved_model_path = "checkpoints/"
pth_name = "saved_model.pth"
saved_checkpoint = torch.load(os.path.join(saved_model_path, pth_name))

# Initialize model and load saved state_dict
# model = ExModel(2).to(device)
model.load_state_dict(saved_checkpoint['state_dict'])


<All keys matched successfully>

In [14]:

test_data = custom_dataset("test")


test_loader = DataLoader(
    test_data,
    batch_size=32
)


def test(model, data_test, loss_function):

    f1_score = 0
    total_loss = 0
    data_iterator = enumerate(data_test)  # take batches
    with torch.no_grad():

        model.eval()  # switch model to evaluation mode
        tq =progress_bar(total=len(data_test))
        tq.set_description('Testing:')


        for _, batch in data_iterator:
            # forward propagation
            images, labels = batch
            # images,labels=images.to(device),labels.to(device)
            

            # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            # model.to(device)
            # images = images.to(device)

            pred = model(images.to(device))
            # pred = model(image.cuda())
            pred = softmax(pred, dim=1)
            # loss = loss_function(pred.cuda(), label.cuda())
            loss = loss_function(pred, labels.to(device))

            # softmax mes 53 netiemiz var onlari ele formta salir ki her biri mehz 1-in nece faizini teskil edir.hamsini ele formta salir ki umumi cemleri 1 olur
            # pred = pred.softmax(dim=1)
            f1_score += metrics(pred, labels.to(device),device)
            # f1_score += metrics(pred, label.cuda())
            total_loss += loss.item()
            tq.update(1)

    # writer.add_scalar("Test mIoU", f1_score/len(data_test), epoch)
    # writer.add_scalar("Test Loss", total_loss/len(data_test), epoch)

    
    tq.close()
    print("F1 score: ", f1_score/len(data_test))
    print("Test Loss: ", total_loss/len(data_test))

    return None



# Evaluate performance on test data
loss = nn.CrossEntropyLoss()
# writer = SummaryWriter()
test(model, test_loader, loss)

check if dataset is balanced: 
712 726 750


Testing:: 100%|██████████| 6/6 [00:00<00:00,  6.09it/s]

F1 score:  tensor(0.8387, device='cuda:0')
Test Loss:  0.7865167359511057


In [ ]:
# bundan asagidaki setirlere fikir vermeyin onemli olan hisse bura qederdir 